In [77]:
import gudhi as gd
from gudhi import representations
import numpy as np
import numpy.random
import matplotlib.pyplot as plt
#import pingouin as pg
from scipy.special import gamma
import scipy.stats as st
from collections import defaultdict
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import paired_distances
from tqdm.notebook import tqdm
import scipy.spatial.distance as spdists
from adaptive.learner.triangulation import circumsphere, fast_2d_circumcircle, fast_3d_circumcircle
from adaptive import domain
import math
from scipy.integrate import nquad
#from curlyBrace import curlyBrace
%matplotlib inline

In [2]:
#scipy.stats uses numpy.random seed
np.random.seed(seed=0)

In [3]:
def sampled_ecc_alpha(pointcloud, n_samples, max_filt_val):
    ac = gd.AlphaComplex(points = pointcloud)
    st = ac.create_simplex_tree()
    filt_values = np.linspace(0,max_filt_val,n_samples)
    ecc_values = np.zeros_like(filt_values)
    simplices = list(st.get_filtration())
    cur_val = 0
    i=0
    for (j,t) in enumerate(filt_values):
        while (i<len(simplices)) and (simplices[i][1]<=t):
            cur_val=cur_val +(-1)**(len(simplices[i][0])-1)
            i = i+1
        #print(t, cur_val)
        ecc_values[j] = cur_val
        
    return ecc_values/np.sqrt(len(pointcloud))

def get_ecc(data, n_samples, max_filt_val):
    eccs = []
    for dat in tqdm(data):
        ec = sampled_ecc_alpha(dat, n_samples, max_filt_val)
        eccs.append(ec)
    return eccs

In [65]:
def h(Y):
    C,_ = circumsphere(Y)
    hull = domain.ConvexHull(Y)
    if hull.encloses(C):
        return 1
    else:
        return 0
    
def h1(Y):
    C,R = circumsphere(Y)
    
    if R<1:
        hull = domain.ConvexHull(Y)
        if hull.encloses(C):
            return 1
        else:
            return 0
            
    else:
        return 0
    
def omega(d):
    return (np.pi**(d/2))/gamma(1+d/2)
    
def Integrand(rv, x, y, lmbd):
    k,d = y.shape
    Y = np.concatenate([[np.zeros(d)],y])
    C,R = circumsphere(Y)
    if R<1:
        hull = domain.ConvexHull(Y)
        if hull.encloses(C):
            return rv.pdf(x)**(k+1)*np.exp(-lmbd*omega(d)*(R**d)*rv.pdf(x))
        else:
            return 0
    else:
        return 0

def gamma(rv, k, lmbd):
    I = 0#integral
    if k == 1:


    return ((lmbd**k)/math.factorial(k+1)) * I

    



In [74]:
rv_norm = st.multivariate_normal()
Integrand(rv_norm,np.array([0,0]),np.array([[0.5,0],[0,0.5]]),1)

array([0.05428649, 0.05428649])